Python で学ぶブロックチェーン


2.3. Python 実装

Python でブロックのヘッダー定義および変数初期化を以下のように行います。

In [ ]:
class Block:
    def __init__(self, index, timestamp, nonce, previous_block_hash, merkle_root, transactions):
        self.index = index  # インデックス
        self.timestamp = timestamp  # タイムスタンプ
        self.nonce = nonce  # ナンス
        self.previous_block_hash = previous_block_hash  # 前のブロックハッシュ値
        self.merkle_root = merkle_root  # マークルルート

トランザクションプールの登録については、以下の通り実装します。キュークラスで transaction_pool を定義し、add_transaction 関数でトランザクションを transaction_pool に PUT しています。

In [ ]:
from queue import Queue

class Blockchain:
    def __init__(self):
        self.transaction_pool = Queue()  # トランザクションプール

    # トランザクションを追加する関数
    def add_transaction(self, transaction):
        self.transaction_pool.put(transaction)

マイニング関数では、transaction_pool からトランザクションを取り出し、リストとしてセットしています。その後、ブロックチェーンとしてブロックを追加する際に、transaction_pool を初期化します。

In [ ]:
# マイニングする関数
def mine_block(self):
    ## ...
    new_block_transactions = list(self.transaction_pool.queue)
    ## ...
    while True:
        ## ...
        # ハッシュ値が要求された難易度より小さい場合、ブロックを追加して終了
        if new_block_hash[:self.difficulty] == '0' * self.difficulty:
            self.transaction_pool = Queue()
            ## ...

4. ハッシュとマークルルート

4.5. Python 実装

4.5.1. ハッシュ

In [ ]:
# SHA256でハッシュ計算する関数
def calculate_hash(self):
    block_string = json.dumps({
        'index': self.index,  # インデックス
        'timestamp': self.timestamp,  # タイムスタンプ
        'nonce': self.nonce,  # ナンス
        'previous_block_hash': self.previous_block_hash,  # 前のブロックハッシュ値
        'merkle_root': self.merkle_root,  # マークルルート
    }, sort_keys=True).encode()

    return hashlib.sha256(block_string).hexdigest()

前のブロックハッシュ値は、マイニングする際に、前のチェーンの block_hash から取得します。

In [ ]:
# マイニングする関数
def mine_block(self):
    # 直前のブロックを取得
    previous_block = self.chain[-1]
    # ...
    new_block_previous_hash = previous_block['block_hash']

4.5.2. マークルルート

マークルルートは以下のアルゴリズムで処理します。

トランザクション数が0の場合は、空文字列のハッシュ値を返す
トランザクション数が1の場合は、そのトランザクションのハッシュ値を返す
要素数が奇数の場合は、最後の要素を複製して偶数にする
2つの要素を組み合わせてハッシュ値を計算し、リストに格納
3.〜4.を再起的に繰り返す。

In [ ]:
# マークルルートを計算する関数
def calculate_merkle_root(self, transactions):
    # 要素数が0の場合は、空文字列のハッシュ値を返す
    if len(transactions) == 0:
        return hashlib.sha256(b'').hexdigest()

    # 要素数が1の場合は、そのトランザクションのハッシュ値を返す
    if len(transactions) == 1:
        return hashlib.sha256(transactions[0].encode()).hexdigest()

    # 要素数が奇数の場合は、最後の要素を複製して偶数にする
    if len(transactions) % 2 == 1:
        transactions.append(transactions[-1])

    # ハッシュ値を格納するリストを初期化
    hashes = []

    # 2つのトランザクションを組み合わせてハッシュ値を計算し、リストに格納する
    for i in range(0, len(transactions), 2):
        transaction_pair = transactions[i] + transactions[i+1]
        hash_value = hashlib.sha256(transaction_pair.encode()).hexdigest()
        hashes.append(hash_value)

    # 再帰的にマークルルートを計算する
    return self.calculate_merkle_root(hashes)

5. ナンスと難易度

In [ ]:
class Blockchain:
    def __init__(self):
        # ...
        self.difficulty = 2  # 難易度
    # ...
    # マイニングする関数
    def mine_block(self):
        # ...
        new_block_nonce = 0
        # ...
        while True:
            # 新しいブロックを作成
            new_block = Block(
                index = new_block_index,
                timestamp = new_block_timestamp,
                nonce = new_block_nonce,
                previous_block_hash = new_block_previous_hash,
                merkle_root = new_block_merkle_root,
                transactions = new_block_transactions
            )
            # ...
            new_block_hash = new_block.calculate_hash()

            # ハッシュ値が要求された難易度より小さい場合、ブロックを追加して終了
            if new_block_hash[:self.difficulty] == '0' * self.difficulty:
                break;

            # nonceを増やして再度試行
            new_block_nonce += 1

6. 電子署名、公開鍵、秘密鍵

9.1. ソースコード全文

In [ ]:
import hashlib
import json
import time
from queue import Queue

# ブロックのクラス
class Block:
    def __init__(self, index, timestamp, nonce, previous_block_hash, merkle_root, transactions):
        self.index = index  # インデックス
        self.timestamp = timestamp  # タイムスタンプ
        self.nonce = nonce  # ナンス
        self.previous_block_hash = previous_block_hash  # 前のブロックハッシュ値
        self.merkle_root = merkle_root  # マークルルート
        self.transactions = transactions  # トランザクション
        self.block_hash = self.calculate_hash()  # ブロックハッシュ値

    # SHA256でハッシュ計算する関数
    def calculate_hash(self):
        block_string = json.dumps({
            'index': self.index,  # インデックス
            'timestamp': self.timestamp,  # タイムスタンプ
            'nonce': self.nonce,  # ナンス
            'previous_block_hash': self.previous_block_hash,  # 前のブロックハッシュ値
            'merkle_root': self.merkle_root,  # マークルルート
        }, sort_keys=True).encode()

        return hashlib.sha256(block_string).hexdigest()

    # マークルルートを計算する関数
    def calculate_merkle_root(self, transactions):
        # 要素数が0の場合は、空文字列のハッシュ値を返す
        if len(transactions) == 0:
            return hashlib.sha256(b'').hexdigest()

        # 要素数が1の場合は、そのトランザクションのハッシュ値を返す
        if len(transactions) == 1:
            return hashlib.sha256(transactions[0].encode()).hexdigest()

        # 要素数が奇数の場合は、最後の要素を複製して偶数にする
        if len(transactions) % 2 == 1:
            transactions.append(transactions[-1])

        # ハッシュ値を格納するリストを初期化
        hashes = []

        # 2つのトランザクションを組み合わせてハッシュ値を計算し、リストに格納する
        for i in range(0, len(transactions), 2):
            transaction_pair = transactions[i] + transactions[i+1]
            hash_value = hashlib.sha256(transaction_pair.encode()).hexdigest()
            hashes.append(hash_value)

        # 再帰的にマークルルートを計算する
        return self.calculate_merkle_root(hashes)

# ブロックチェーンのクラス
class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]  # チェーン
        self.transaction_pool = Queue()  # トランザクションプール
        self.difficulty = 2  # 難易度

    # ジェネシスブロックを生成する関数
    def create_genesis_block(self):
        genesis_block = Block(
            index=0,
            timestamp=time.time(),
            nonce=0,
            previous_block_hash='',
            merkle_root='',
            transactions=[],
        )
        return genesis_block.__dict__

    # トランザクションを追加する関数
    def add_transaction(self, transaction):
        self.transaction_pool.put(transaction)

    # マイニングする関数
    def mine_block(self):
        # 直前のブロックを取得
        previous_block = self.chain[-1]

        # 新しいブロックのデータを設定
        new_block_index = previous_block['index'] + 1
        new_block_timestamp = time.time()
        new_block_nonce = 0
        new_block_previous_hash = previous_block['block_hash']
        new_block_merkle_root = ''
        new_block_transactions = list(self.transaction_pool.queue)

        while True:
            # 新しいブロックを作成
            new_block = Block(
                index = new_block_index,
                timestamp = new_block_timestamp,
                nonce = new_block_nonce,
                previous_block_hash = new_block_previous_hash,
                merkle_root = new_block_merkle_root,
                transactions = new_block_transactions
            )
            new_block.merkle_root = new_block.calculate_merkle_root(new_block_transactions),
            new_block_hash = new_block.calculate_hash()

            # ハッシュ値が要求された難易度より小さい場合、ブロックを追加して終了
            if new_block_hash[:self.difficulty] == '0' * self.difficulty:
                self.chain.append(new_block.__dict__)
                self.transaction_pool = Queue()

                return new_block.__dict__

            # nonceを増やして再度試行
            new_block_nonce += 1

In [ ]:
# 1.トランザクションAliceを追加　ジェネシスのみの確認
blockchain=Blockchain()
blockchain.add_transaction("Alice")
print(blockchain.chain)

[{'index': 0, 'timestamp': 1690594577.2373264, 'nonce': 0, 'previous_block_hash': '', 'merkle_root': '', 'transactions': [], 'block_hash': 'c6fdf3d57ff0e1f8363c9c720b140a4169931a03860857b317a27a4644c40dae'}]


In [ ]:
# 2.Aliceを含むトランザクションをブロック1に処理
blockchain.mine_block()

{'index': 1,
 'timestamp': 1690594583.4873512,
 'nonce': 22,
 'previous_block_hash': 'c6fdf3d57ff0e1f8363c9c720b140a4169931a03860857b317a27a4644c40dae',
 'merkle_root': ('3bc51062973c458d5a6f2d8d64a023246354ad7e064b1e4e009ec8a0699a3043',),
 'transactions': ['Alice'],
 'block_hash': '9d8633a3678a182ed607d4c02caec6b5e7ec29a3196e2a722c8b06e7b775c741'}

In [ ]:
# 3.ブロック１の確認
print(blockchain.chain)

[{'index': 0, 'timestamp': 1690594577.2373264, 'nonce': 0, 'previous_block_hash': '', 'merkle_root': '', 'transactions': [], 'block_hash': 'c6fdf3d57ff0e1f8363c9c720b140a4169931a03860857b317a27a4644c40dae'}, {'index': 1, 'timestamp': 1690594583.4873512, 'nonce': 22, 'previous_block_hash': 'c6fdf3d57ff0e1f8363c9c720b140a4169931a03860857b317a27a4644c40dae', 'merkle_root': ('3bc51062973c458d5a6f2d8d64a023246354ad7e064b1e4e009ec8a0699a3043',), 'transactions': ['Alice'], 'block_hash': '9d8633a3678a182ed607d4c02caec6b5e7ec29a3196e2a722c8b06e7b775c741'}]


In [ ]:
# 4.トランザクションBobとGraceを追加して、確認
blockchain.add_transaction("Bob")
blockchain.add_transaction("Grace")
print(blockchain.chain)

[{'index': 0, 'timestamp': 1690594577.2373264, 'nonce': 0, 'previous_block_hash': '', 'merkle_root': '', 'transactions': [], 'block_hash': 'c6fdf3d57ff0e1f8363c9c720b140a4169931a03860857b317a27a4644c40dae'}, {'index': 1, 'timestamp': 1690594583.4873512, 'nonce': 22, 'previous_block_hash': 'c6fdf3d57ff0e1f8363c9c720b140a4169931a03860857b317a27a4644c40dae', 'merkle_root': ('3bc51062973c458d5a6f2d8d64a023246354ad7e064b1e4e009ec8a0699a3043',), 'transactions': ['Alice'], 'block_hash': '9d8633a3678a182ed607d4c02caec6b5e7ec29a3196e2a722c8b06e7b775c741'}]


In [ ]:
# 5.マイニングする　ブロック２の処理
blockchain.mine_block()

{'index': 2,
 'timestamp': 1690594594.504145,
 'nonce': 56,
 'previous_block_hash': '9d8633a3678a182ed607d4c02caec6b5e7ec29a3196e2a722c8b06e7b775c741',
 'merkle_root': ('8eb45a82f786577388e204d82ffbedc61fcbfaa7fc31a8fae6e0c163e35d5dd5',),
 'transactions': ['Bob', 'Grace'],
 'block_hash': '9b0a86fa34c0dad4042188e8908ba2154e6497ab576923e907b2498aab02807c'}

In [ ]:
# 6.ブロック２の確認
print(blockchain.chain)

[{'index': 0, 'timestamp': 1690594577.2373264, 'nonce': 0, 'previous_block_hash': '', 'merkle_root': '', 'transactions': [], 'block_hash': 'c6fdf3d57ff0e1f8363c9c720b140a4169931a03860857b317a27a4644c40dae'}, {'index': 1, 'timestamp': 1690594583.4873512, 'nonce': 22, 'previous_block_hash': 'c6fdf3d57ff0e1f8363c9c720b140a4169931a03860857b317a27a4644c40dae', 'merkle_root': ('3bc51062973c458d5a6f2d8d64a023246354ad7e064b1e4e009ec8a0699a3043',), 'transactions': ['Alice'], 'block_hash': '9d8633a3678a182ed607d4c02caec6b5e7ec29a3196e2a722c8b06e7b775c741'}, {'index': 2, 'timestamp': 1690594594.504145, 'nonce': 56, 'previous_block_hash': '9d8633a3678a182ed607d4c02caec6b5e7ec29a3196e2a722c8b06e7b775c741', 'merkle_root': ('8eb45a82f786577388e204d82ffbedc61fcbfaa7fc31a8fae6e0c163e35d5dd5',), 'transactions': ['Bob', 'Grace'], 'block_hash': '9b0a86fa34c0dad4042188e8908ba2154e6497ab576923e907b2498aab02807c'}]
